# Train a crash detector on the cloud using YoloV3 and Darknet 
The current notebook gives a step by step guide to train an object detector using darknet framework and YoloV3 model. We'll make use of the GPU offered by Google Colab Platform to accelerate the training phase.

# Step 1: Mount Darknet on the Cloud

The following cells clone the darknet repository and modifies the config file Makefile to activate OpenCV which allows feeding video footage to the algorithm and accelerates the training phase of the model.

In [ ]:
# cloning darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 13567 (delta 1), reused 1 (delta 0), pack-reused 13559
Receiving objects: 100% (13567/13567), 12.20 MiB | 11.74 MiB/s, done.
Resolving deltas: 100% (9254/9254), done.


In [ ]:
# activate GPU and OpenCV
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
# verify CUDA version
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
# compile darknet
!make

# Step 2: helper functions 
These function will be used later to simplify the training process and testing the model.

In [ ]:
#imShow function takes as input a path to an image and shows the image using matplotlib
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# function to upload files to google colab
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# function to download files from google colab
def download(path):
  from google.colab import files
  files.download(path)

# Step 3 : Mount Google Drive on Google Colab
Here we will connect our drive account to google colab in order to save the model checkpoints during training and the best weights after training as well as importing training data and any other useful files.


In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/content
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# create a sympolic link to remove spaces and simplify access to the drive folders (optional)
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# darknet directory
%cd darknet

/content/darknet


In [ ]:
# Copier un fichier à partir du drive vers le répertoire darknet
# Copy a pretrained model to accelerate learning
#(see https://www.tensorflow.org/tutorials/images/transfer_learning) 
!cp /mydrive/yolov3/darknet53.conv.74  ../

# Step 4: Create a dataset
In order to train our model we'll need a dataset of images/videos related to the problem we're trying to solve. 
There two way to get a build dataset.
- **Method 1:** Using an automation tool or script to download labelled images from datasets websites such as : ImageNet, Open Image Dataset, Kaggle...etc.
- **Method 2:** Download and label manually the images using a tool called LabelImg.

We'll be using the second method since our target dataset doesn't exist in such websites. 

# Step 5: Move Dataset to google colab's current instance
We'll zip our dataset and download it from the drive

In [ ]:

!ls /mydrive/yolov3_crash

backup		   generate_train.py  obj.data	 yolov3_custom2.cfg
darknet53.conv.74  obj2.zip	      obj.names


In [ ]:
# copy the zip file to the current directory of google colab's instance
!cp /mydrive/yolov3_crash/obj2.zip ../

In [ ]:
# unzip the file
!unzip ../obj2.zip -d data/

# Step 6: Edit config file of YoloV3
Edit the following files .cfg, obj.data, obj.names and train.txt.

## i) .cfg file
We edit the .cfg file to adapt it to our needs and put it in darknet directory. 

In [ ]:
!cp /mydrive/yolov3_crash/yolov3_custom2.cfg ./cfg


## ii) obj.names and obj.data 
**obj.names** file contains the list of classes we're looking to detect using YoloV3. In our case we choose one perticular scenario which is car rollovers.

**obj.data** file contains the total number of classes we're training our model on and file paths to obj.names and train.txt files. It also contains the path to backup files where we'll save our training progress and checkpoints.

In [ ]:
# move files obj.names and obj.data from the drive to google colab's instance
!cp /mydrive/yolov3_crash/obj.names ./data
!cp /mydrive/yolov3_crash/obj.data  ./data

## iii) Generate train.txt
Ce fichier de configuration contient les chemins relatives de tous les images de notre dataset d'entrainement.
train.txt file contains the relative paths to the images of our training dataset.
We built a small python script to automate the creation of this file.

In [ ]:
# move generate_train.py file to google colab
!cp /mydrive/yolov3_crash/generate_train.py ./


In [ ]:
!python generate_train.py

In [ ]:
#verify that file was properly generated
!ls data/
!cat data/train.txt | sort -n

# Step 7: Train !
Now after finishing all configurations, the last step is to run the following darkent command
```
!./darknet detector train <path to obj.data> <path to cfg file> darknet53.conv.74 -dont_show
```
-dont show option stops the algorithme from showing the training chart, since it generates an error on colab.  

In [ ]:

!./darknet detector train data/obj.data cfg/yolov3_custom2.cfg darknet53.conv.74 -map -dont_show

# Step 8: run the detection
We can finally test our model on images, or download the best wieghts and test the model locally on videos or on a realtime webcam.

In [ ]:
# switch the configuration file to test mode
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov3_custom2.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3_custom2.cfg
%cd ..


In [ ]:
# test the model on an image
!./darknet detector test data/obj.data cfg/yolov3_custom2.cfg /mydrive/yolov3/backup/yolov3_custom2_last.weights /mydrive/yolov3/images/rollovertest.jpg -thresh 0.3
imShow('predictions.jpg')